# Query the data

## Get keys and urls

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)
print(OPENAI_API_KEY)

if(WEAVIATE_URL == "UPDATE_ME_WEAVIATE_URL"):
    raise Exception("Please update .env and Restart the notebook (see Restart button, next to Run All)")

https://3gr2a6vetrmhdtkhud1ryg.c0.europe-west3.gcp.weaviate.cloud
vkZQeDJjZKaPKIOcQDssGPARPsAjzV9YpqsH
sk-cWzmxOAnwcNG3ii1cmm44maziAEovRo4vT5f-aGy3dT3BlbkFJ8QHwsLfRWkNVnXhpyCAuxsE3qRMBfAFkKAh5xKibUA


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

/home/vscode/.local/lib/python3.12/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


True

### Helper function

In [3]:
import json
def print_properties(item):
    print(
        json.dumps(
            item.properties,
            indent=2, sort_keys=True, default=str
        )
    )

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [4]:
# TODO: get the Wiki collection
wiki = client.collections.get("Wiki")

# TODO: run a near text query, search for musical instruments, with limit 5
response = wiki.query.near_text("musical instruments", limit=25)

for item in response.objects:
    print_properties(item)

{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wiki/Tuba",
  "wiki_id": "20231101.simple_11118_0"
}
{
  "text": "Church music has varied enormously during the history of Christianity as different churches kept changing their ideas about what part music should have in religious worship.  Most church music is based on singing.  Music written for church choirs mostly used the words of the liturgy (the words used in services).  The organ is the most important musical instrument in church music, although from time to time many other instruments have been used as well.",
  "title": "Church music",
  "url": "https://simple.wikipedia.org/wiki/Church%20music",
  "wiki_id": "20231101.simple_93102_1"
}
{
  "text": "The orchestra needs 3 flu

In [5]:
from weaviate.classes.query import MetadataQuery

# wiki = client.collections.get(WIKI_NAME)

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    # TODO: add MetadataQuery - request distance
    return_metadata=MetadataQuery(distance=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wiki/Tuba",
  "wiki_id": "20231101.simple_11118_0"
}
0.5253647565841675
{
  "text": "Church music has varied enormously during the history of Christianity as different churches kept changing their ideas about what part music should have in religious worship.  Most church music is based on singing.  Music written for church choirs mostly used the words of the liturgy (the words used in services).  The organ is the most important musical instrument in church music, although from time to time many other instruments have been used as well.",
  "title": "Church music",
  "url": "https://simple.wikipedia.org/wiki/Church%20music",
  "wiki_id": "20231101.simple_93102_1"
}
0.5491465330123901


### Autocut

Return groups of results based on the quality/distance jumps

In [13]:
from weaviate.classes.query import MetadataQuery

wiki = client.collections.get("wiki")

response = wiki.query.near_text(
    query="musical instruments",
    # TODO: use auto_limit instead if limit, set it to 1
    auto_limit=2,
    distance=0.561,

    return_metadata=MetadataQuery(distance=True)
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

Returned object count: 2
{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wiki/Tuba",
  "wiki_id": "20231101.simple_11118_0"
}
0.5253647565841675
{
  "text": "Church music has varied enormously during the history of Christianity as different churches kept changing their ideas about what part music should have in religious worship.  Most church music is based on singing.  Music written for church choirs mostly used the words of the liturgy (the words used in services).  The organ is the most important musical instrument in church music, although from time to time many other instruments have been used as well.",
  "title": "Church music",
  "url": "https://simple.wikipedia.org/wiki/Church%20music",
  "wiki_id": "20231101.simple_93102

## Filters

### Fetch with filters

In [16]:
from weaviate.classes.query import Filter

# wiki = client.collections.get(WIKI_NAME)

response = wiki.query.fetch_objects(
    limit=5,
    # TODO: filter by property title, search for something like music
    filters=Filter.by_property("title")
)

for item in response.objects:
    print_properties(item)

WeaviateInvalidInputError: Invalid input provided: Argument 'filters' must be one of: [<class 'weaviate.collections.classes.filters._Filters'>, None], but got <class 'weaviate.collections.classes.filters._FilterByProperty'>.

In [14]:
from weaviate.classes.query import Filter

response = wiki.query.fetch_objects(
    limit=5,
    # NOTE: you can use & as AND operator and | as OR operator
    filters=Filter.by_property("title").like("m*ic") & Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

{
  "text": "By its very nature, the evidence for this idea comes mainly from palaeontology. It is not claimed that this pattern is universal, but it is common. There are now a wide range of examples from many different taxa.",
  "title": "Mosaic evolution",
  "url": "https://simple.wikipedia.org/wiki/Mosaic%20evolution",
  "wiki_id": "20231101.simple_290228_4"
}
{
  "text": "In 1886, Adolf Mayer first described the tobacco mosaic disease that could be transferred between plants, similar to bacterial infections.",
  "title": "Tobacco mosaic virus",
  "url": "https://simple.wikipedia.org/wiki/Tobacco%20mosaic%20virus",
  "wiki_id": "20231101.simple_528724_2"
}
{
  "text": "\"Magic\" by Pilot is used, its 3rd use in a movie soundtrack after Disney's Herbie Fully Loaded and Happy Gilmore",
  "title": "The Magic Roundabout (movie)",
  "url": "https://simple.wikipedia.org/wiki/The%20Magic%20Roundabout%20%28movie%29",
  "wiki_id": "20231101.simple_117801_2"
}
{
  "text": "What enables the Kr

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [15]:
from weaviate.classes.query import Filter

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    filters=Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wiki/Tuba",
  "wiki_id": "20231101.simple_11118_0"
}
{
  "text": "The orchestra needs 3 flutes (one doubling piccolo), 3 oboes (one doubling cor anglais), 3 clarinets (one doubling bass clarinet), 3 bassoons (one doubling contrabassoon), 4 French horns, 3 trumpets, 3 trombones, tuba, timpani, side drum, bass drum, cymbals, triangle, tamtam, 2 harps and strings",
  "title": "Concerto for Orchestra (Bart\u00f3k)",
  "url": "https://simple.wikipedia.org/wiki/Concerto%20for%20Orchestra%20%28Bart%C3%B3k%29",
  "wiki_id": "20231101.simple_193305_4"
}
{
  "text": "The simplest type of mechanical aid is the use of wooden \"fingers\" that can be closed to press the string down and fret the C#

## Keyword Search

[Docs - keyword/bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [19]:
# TODO: use bm25 query, search for musical instruments, set limit to 5

response = wiki.query.bm25("Music", limit=5)

for item in response.objects:
    print_properties(item)

{
  "text": "Solf\u00e8ge (sometimes called solfa) is the way tones are named. It was made in order to give a name to the several tones and pitches. For example, the eight basic notes \"Do, Re, Mi, Fa, So, La, Ti, Do\" are just the names of the eight notes that confirm the major scale.",
  "title": "Music",
  "url": "https://simple.wikipedia.org/wiki/Music",
  "wiki_id": "20231101.simple_472_41"
}
{
  "text": "The range of a musical instrument is the distance from the lowest to the highest pitch it can play. For a singing voice, this is known as vocal range.",
  "title": "Range (music)",
  "url": "https://simple.wikipedia.org/wiki/Range%20%28music%29",
  "wiki_id": "20231101.simple_84201_0"
}
{
  "text": "Double sharps also exist; these have the symbol  and raise a note by two semitones, or a whole step. Less often there are half or three-quarter sharps.",
  "title": "Sharp (music)",
  "url": "https://simple.wikipedia.org/wiki/Sharp%20%28music%29",
  "wiki_id": "20231101.simple_233800_

In [21]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    # TODO: add query properties for "text" and "title"
    query_properties=["title", "text"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

{
  "text": "Thousands of years ago people used the horns of dead animals for musical instruments (see horn (postal horn)). In later times, they were used for carrying gunpowder.",
  "title": "Horn (anatomy)",
  "url": "https://simple.wikipedia.org/wiki/Horn%20%28anatomy%29",
  "wiki_id": "20231101.simple_43954_2"
}
4.7339887619018555
{
  "text": "MTV Unplugged is a TV series. It showcases many popular musical artists usually playing acoustic instruments. It received the George Foster Peabody Award and 3 Primetime Emmy nominations.",
  "title": "MTV Unplugged",
  "url": "https://simple.wikipedia.org/wiki/MTV%20Unplugged",
  "wiki_id": "20231101.simple_474372_0"
}
4.383030414581299
{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wi

In [22]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    query_properties=["text", "title^3"], # makes titel 3x more important
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

{
  "text": "Thousands of years ago people used the horns of dead animals for musical instruments (see horn (postal horn)). In later times, they were used for carrying gunpowder.",
  "title": "Horn (anatomy)",
  "url": "https://simple.wikipedia.org/wiki/Horn%20%28anatomy%29",
  "wiki_id": "20231101.simple_43954_2"
}
4.7339887619018555
{
  "text": "Cats premiered at the New London Theatre in the West End on 11 May 1981. It closed on 11 May 2002 after 8,949 performances. It opened on Broadway on 7 October 1982 at the Winter Garden Theatre. It closed on 10 September 2000, after 7,485 performances.",
  "title": "Cats (musical)",
  "url": "https://simple.wikipedia.org/wiki/Cats%20%28musical%29",
  "wiki_id": "20231101.simple_70788_1"
}
4.688827991485596
{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Musical tun

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [24]:
# TODO: use hybrid query, search for musical instruments, set alpha to 0.7, and limit to 5
response = wiki.query.hybrid("musical instruments", alpha=0.7)

for item in response.objects:
    print_properties(item)

{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wiki/Tuba",
  "wiki_id": "20231101.simple_11118_0"
}
{
  "text": "Thousands of years ago people used the horns of dead animals for musical instruments (see horn (postal horn)). In later times, they were used for carrying gunpowder.",
  "title": "Horn (anatomy)",
  "url": "https://simple.wikipedia.org/wiki/Horn%20%28anatomy%29",
  "wiki_id": "20231101.simple_43954_2"
}
{
  "text": "Church music has varied enormously during the history of Christianity as different churches kept changing their ideas about what part music should have in religious worship.  Most church music is based on singing.  Music written for church choirs mostly used the words of the liturgy (the words used in servi

### Hybrid - select properties

In [26]:
response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    # TODO: add query properties for "title"
    query_properties=["title"]
)

for item in response.objects:
    print_properties(item)

{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wiki/Tuba",
  "wiki_id": "20231101.simple_11118_0"
}
{
  "text": "Church music has varied enormously during the history of Christianity as different churches kept changing their ideas about what part music should have in religious worship.  Most church music is based on singing.  Music written for church choirs mostly used the words of the liturgy (the words used in services).  The organ is the most important musical instrument in church music, although from time to time many other instruments have been used as well.",
  "title": "Church music",
  "url": "https://simple.wikipedia.org/wiki/Church%20music",
  "wiki_id": "20231101.simple_93102_1"
}
{
  "text": "The orchestra needs 3 flu

### Hybrid - Explain score

In [25]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    query_properties=["title"],
    return_metadata=MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)
    print(item.metadata.explain_score)

{
  "text": "The tuba is the biggest of all the brass musical instruments. They are the newest part of the symphony orchestra, first showing up in the mid-19th century. Most orchestras have a tuba now. The standard tuba has about 16 feet of tubes.",
  "title": "Tuba",
  "url": "https://simple.wikipedia.org/wiki/Tuba",
  "wiki_id": "20231101.simple_11118_0"
}
0.699999988079071

Hybrid (Result Set vector,hybridVector) Document 2e92b095-929d-5949-9423-25b0df1807af: original score 0.47463524, normalized score: 0.7
{
  "text": "Church music has varied enormously during the history of Christianity as different churches kept changing their ideas about what part music should have in religious worship.  Most church music is based on singing.  Music written for church choirs mostly used the words of the liturgy (the words used in services).  The organ is the most important musical instrument in church music, although from time to time many other instruments have been used as well.",
  "title": "

## Close the client

In [27]:
client.close()